In [20]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


Workspace name: quick-starts-ws-130032
Azure region: southcentralus
Subscription id: 26806ae2-7725-4970-9c73-e6b2c7c706c1
Resource group: aml-quickstarts-130032


In [21]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

#name for cpu cluster
amlcompute_cluster_name = "cpu-cluster"

#check for existance of cluster
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, using it ...')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes = 4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
aml_compute.wait_for_completion(show_output=True)


Found existing cluster, using it ...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
    "--C": uniform(0.01, 1),
    "--max_iter": choice([100, 200, 300, 500])
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=3)
import shutil
if "training" not in os.listdir():
    os.mkdir("./training")
os.makedirs("./training", exist_ok=True)
shutil.copy("train.py","./training")
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './training',
              compute_target=aml_compute,
              entry_script='train.py',
              pip_packages=["pyarrow>=0.12.0", "pyspark"])
primary_metric_name = "Accuracy"
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name=primary_metric_name,
                                     primary_metric_goal=primary_metric_goal,
                                     policy=policy,
                                     max_total_runs=20,
                                     max_concurrent_runs=3,
                                     )

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperopt_run = exp.submit(hyperdrive_config, tag={'type':'hyperdrive_opt'})
RunDetails(hyperopt_run).show()
### YOUR CODE HERE ###

In [ ]:
import joblib
# Get your best run and save the model from that run.
hyperopt_run.wait_for_completion(show_output=True)
assert (hyperopt_run.get_status() == "Completed")
best_run = hyperopt_run.get_best_run_by_primary_metric()

best_run.register_model(model_name="bankmarketing_hyperdrive", model_path="outputs/bankmarketing_hyperdrive.joblib")

In [28]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
bankmarketing_data = TabularDatasetFactory.from_delimited_files(url)

In [29]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(bankmarketing_data)

In [47]:
from azureml.train.automl import AutoMLConfig


# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
x["y"] = y # add target to training data
ds = ws.get_default_datastore()
input_data = TabularDatasetFactory.register_pandas_dataframe(dataframe=x, target=ds, name="bankmarketing" )
automl_settings = {
    "n_crossvalidations": 4,
    "enable_early_stopping": True,
    "primary_metric": "accuracy",
    
}
automl_config = AutoMLConfig(
    path=".",
    experiment_timeout_minutes=30,
    task="classification",
    compute_target=aml_compute,
    training_data=input_data,
    label_column_name="y",
    automl_settings)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/7879b917-e548-497c-a2ff-e45ef73c8366/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [48]:
# Submit your automl run
automl_run = exp.submit(automl_config, tag={'type':'automl'})
automl_run.wait_for_completion(show_output=True)

Running on remote.


In [ ]:
# Retrieve and save your best automl model.

assert (automl_run.get_status() == "Completed")
best_run, best_model = automl_run.get_output()
joblib.dump(best_model, filename="outputs/automl_model.joblib")
best_run.register(name="bankmarketing_automl", file_path="./outputs/")
### YOUR CODE HERE ###